In [2]:
def value_iteration(P, R, gamma=0.9, epsilon=1e-6):
    n_states, n_actions = R.shape[1], R.shape[0]
    V = np.zeros(n_states)
    while True:
        delta = 0
        for s in range(n_states):
            v = V[s]
            V[s] = max(sum(P[a, s, s1] * (R[a, s] + gamma * V[s1]) for s1 in range(n_states)) for a in range(n_actions))
            delta = max(delta, abs(v - V[s]))
        if delta < epsilon:
            break
    policy = np.argmax([[sum(P[a, s, s1] * (R[a, s] + gamma * V[s1]) for s1 in range(n_states)) for a in range(n_actions)] for s in range(n_states)], axis=1)
    return policy, V

Optimal Policy (Value Iteration): [0 0 0]
Value Function (Value Iteration): [7.57356278 7.77248457 8.25188349]
Optimal Policy (Policy Iteration): [0 0 0]
Value Function (Policy Iteration): [7.57356346 7.77248525 8.25188414]
Optimal Policy (Q-Learning): [1 0 0]
Q-Table (Q-Learning): [[5.21292141 6.82114534]
 [7.39741234 6.76682418]
 [9.24107359 6.68600706]]


In [3]:
import numpy as np
import random

### MDP Algorithms ###
def value_iteration(P, R, gamma=0.9, epsilon=1e-6):
    n_states, n_actions = R.shape[1], R.shape[0]
    V = np.zeros(n_states)
    while True:
        delta = 0
        for s in range(n_states):
            v = V[s]
            V[s] = max(sum(P[a, s, s1] * (R[a, s] + gamma * V[s1]) for s1 in range(n_states)) for a in range(n_actions))
            delta = max(delta, abs(v - V[s]))
        if delta < epsilon:
            break
    policy = np.argmax([[sum(P[a, s, s1] * (R[a, s] + gamma * V[s1]) for s1 in range(n_states)) for a in range(n_actions)] for s in range(n_states)], axis=1)
    return policy, V

def policy_iteration(P, R, gamma=0.9, epsilon=1e-6):
    num_states, num_actions = R.shape[1], R.shape[0]
    policy = np.zeros(num_states, dtype=int)
    V = np.zeros(num_states)
    while True:
        while True:
            delta = 0
            for s in range(num_states):
                v = V[s]
                V[s] = sum(P[policy[s], s, s1] * (R[policy[s], s] + gamma * V[s1]) for s1 in range(num_states))
                delta = max(delta, abs(v - V[s]))
            if delta < epsilon:
                break
            policy_stable = True
            for s in range(num_states):
                old_action = policy[s]
                policy[s] = np.argmax([sum(P[a, s, s1] * (R[a, s] + gamma * V[s1]) for s1 in range(num_states)) for a in range(num_actions)])
                if old_action != policy[s]:
                    policy_stable = False
            if policy_stable:
                break
    return policy, V

def q_learning(P, R, gamma=0.9, alpha=0.1, epsilon=0.1, episodes=1000):
    num_states, num_actions = R.shape[1], R.shape[0]
    Q = np.zeros((num_states, num_actions))
    for _ in range(episodes):
        state = random.choice(range(num_states))
        while True:
            if random.uniform(0, 1) < epsilon:
                action = random.choice(range(num_actions))
            else:
                action = np.argmax(Q[state])
            next_state = np.argmax(P[action, state])
            reward = R[action, state]
            best_next_action = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state, best_next_action]
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error
            if state == next_state:
                break
            state = next_state
    policy = np.argmax(Q, axis=1)
    return policy, Q

### Utility Functions ###

def validate_transition_matrix(P):
    assert np.allclose(P.sum(axis=2), 1), "Transition probabilities must sum to 1."

def validate_reward_matrix(R, P):
    assert R.shape == P.shape[:2], "Reward matrix dimensions must match the transition matrix."

def generate_random_mdp(num_states, num_actions):
    P = np.zeros((num_actions, num_states, num_states))
    for a in range(num_actions):
        for s in range(num_states):
            P[a, s, :] = np.random.dirichlet(np.ones(num_states))
    R = np.random.rand(num_actions, num_states)
    return P, R

### Example Usage ###

# Generate a random MDP
num_states = 3
num_actions = 2
P, R = generate_random_mdp(num_states, num_actions)

# Validate the MDP
validate_transition_matrix(P)
validate_reward_matrix(R, P)

# Solve the MDP using Value Iteration
policy_vi, V_vi = value_iteration(P, R)
print("Optimal Policy (Value Iteration):", policy_vi)
print("Value Function (Value Iteration):", V_vi)

# Solve the MDP using Policy Iteration
policy_pi, V_pi = policy_iteration(P, R)
print("Optimal Policy (Policy Iteration):", policy_pi)
print("Value Function (Policy Iteration):", V_pi)

# Solve the MDP using Q-Learning
policy_ql, Q_ql = q_learning(P, R)
print("Optimal Policy (Q-Learning):", policy_ql)
print("Q-Table (Q-Learning):", Q_ql)

Optimal Policy (Value Iteration): [0 1 1]
Value Function (Value Iteration): [6.22147264 5.98458936 6.18126588]
